In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import sys
sys.path.append('/content/drive/My Drive/sentiment_analysis_project')

In [ ]:
from pathlib import Path
import cv2
import dlib
import sys
import numpy as np
import argparse
from contextlib import contextmanager
from wide_resnet import WideResNet
from keras.utils.data_utils import get_file
from keras.models import load_model
from keras.preprocessing.image import img_to_array

classifier = load_model('/content/drive/My Drive/sentiment_analysis_project/pretrained_models/emotion_little_vgg_2.h5')
face_classifier = cv2.CascadeClassifier('/content/drive/My Drive/sentiment_analysis_project/Haarcascades/haarcascade_frontalface_default.xml')
pretrained_model = "weights2.hdf5"

### Testing our Emotion, Age and Gender Detector - Using a Video from Path


In [ ]:
modhash = 'fbe63257a054c1c5466cfd7bf14646d6'
emotion_classes = {0: 'Angry', 1: 'Fear', 2: 'Happy', 3: 'Neutral', 4: 'Sad', 5: 'Surprise'}

def face_detector(img):
    # Convert image to grayscale for faster detection
    gray = cv2.cvtColor(img.copy(),cv2.COLOR_BGR2GRAY)
    faces = face_classifier.detectMultiScale(gray, 1.3, 5)
    if faces is ():
        return False ,(0,0,0,0), np.zeros((1,48,48,3), np.uint8), img
    
    allfaces = []   
    rects = []
    for (x,y,w,h) in faces:
        cv2.rectangle(img,(x,y),(x+w,y+h),(255,0,0),2)
        roi = img[y:y+h, x:x+w]
        allfaces.append(roi)
        rects.append((x,w,y,h))
    return True, rects, allfaces, img

def preprocess_input(x, v2=True):
    x = x.astype('float32')
    x = x / 255.0
    if v2:
        x = x - 0.5
        x = x * 2.0
    return x

# Define our model parameters
depth = 16
k = 8
weight_file = None
margin = 0.4
image_dir = None


# load model and weights
img_size = 64
model = WideResNet(img_size, depth=depth, k=k)()
model.load_weights('/content/drive/My Drive/sentiment_analysis_project/pretrained_models/weights2.hdf5')

gender_classifier = load_model("/content/drive/My Drive/sentiment_analysis_project/pretrained_models/gender/simple_CNN.81-0.96.hdf5", compile=False)
gender_target_size = gender_classifier.input_shape[1:3]


# Initialize Video
cap = cv2.VideoCapture("/content/drive/My Drive/sentiment_analysis_project/one-millionth-customer-prank-50-central.mp4")


frame_width = int(cap.get(3))
frame_height = int(cap.get(4))

fps = cap.get(cv2.CAP_PROP_FPS)
print(fps)
out = cv2.VideoWriter('/content/drive/My Drive/tryVideo_output3.avi',cv2.VideoWriter_fourcc('M','J','P','G'), fps, (frame_width,frame_height))

from IPython.display import clear_output, Image
import base64

while True:
    ret, frame = cap.read()
    ret, rects, faces, image = face_detector(frame)
    preprocessed_faces_ag = []
    preprocessed_faces_emo = []
    
    predicted_genders = []
    if ret:
        for (i,face) in enumerate(faces):
          
            face_ag = cv2.resize(face, (64, 64), interpolation = cv2.INTER_AREA)
            preprocessed_faces_ag.append(face_ag)

            face_ag = cv2.resize(face, gender_target_size)
                                 #, interpolation = cv2.INTER_AREA)
            face_ag = cv2.cvtColor(face_ag, cv2.COLOR_BGR2RGB)
            face_ag = np.expand_dims(face_ag, 0)
            face_ag = preprocess_input(face_ag, False)
            predicted_genders.append(gender_classifier.predict(face_ag))
            #preprocessed_faces_ag.append(face_ag)

            face_gray_emo = cv2.cvtColor(face, cv2.COLOR_BGR2GRAY)
            face_gray_emo = cv2.resize(face_gray_emo, (48, 48), interpolation = cv2.INTER_AREA)
            face_gray_emo = face_gray_emo.astype("float") / 255.0
            face_gray_emo = img_to_array(face_gray_emo)
            face_gray_emo = np.expand_dims(face_gray_emo, axis=0)
            preprocessed_faces_emo.append(face_gray_emo)
            
        # make a prediction for Age and Gender
        results = model.predict(np.array(preprocessed_faces_ag))
        #predicted_genders = results[0]
        
        #print("OK")
        #print(predicted_genders)
        
        ages = np.arange(0, 101).reshape(101, 1)
        predicted_ages = results[1].dot(ages).flatten()

        # make a prediction for Emotion 
        emo_labels = []
        for (i, face) in enumerate(faces):
            preds = classifier.predict(preprocessed_faces_emo[i])[0]
            emo_labels.append(emotion_classes[preds.argmax()])
        
        # draw results, for Age and Gender
        for (i, face) in enumerate(faces):
            #print(predicted_genders[i][0][0])
            label = "{}, {}, {}".format(int(predicted_ages[i]),
                                        "F" if predicted_genders[i][0][0] > 0.5 else "M",
                                        emo_labels[i])
            
        #Overlay our detected emotion on our pic
        for (i, face) in enumerate(faces):
            label_position = (rects[i][0] + int((rects[i][1]/2)), abs(rects[i][2] - 10))
            cv2.putText(image, label, label_position , cv2.FONT_HERSHEY_PLAIN,1, (0,255,0), 2)

    #cv2.imshow(image)
    out.write(image)

    if cv2.waitKey(1) == 13: #13 is the Enter Key
        break

cap.release()
out.release()
cv2.destroyAllWindows()      

29.97002997002997
